In [58]:
using Revise, LazySets

## Idea

In [ ]:
### Signatures:

```
function overapproximate(r::Rectification{N, ZT}, ::Type{<:Zonotope}) where {N, ZT<:AbstractZonotope{N}}
   ...
end
```

```
function overapproximate!(r::Rectification{N, ZT}, Zout::Zonotope{N}, ::Type{<:Zonotope}) where {N, ZT<:AbstractZonotope{N}}
   ...
end
```

In [3]:
Z = rand(Zonotope)
r = Rectification(Z)

Rectification{Float64,Zonotope{Float64,Array{Float64,1},Array{Float64,2}}}(Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([-0.6574585874306136, 0.7654088848008658], [-0.4265028103350649 -1.5366076678870937 0.04633699523455334 -0.9653325008567981; 0.12684187980863493 -0.15352687306547236 1.4104921471862892 0.43648240351708767]), LazySets.RectificationCache{Float64}(nothing, false))

In [5]:
r.X

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([-0.6574585874306136, 0.7654088848008658], [-0.4265028103350649 -1.5366076678870937 0.04633699523455334 -0.9653325008567981; 0.12684187980863493 -0.15352687306547236 1.4104921471862892 0.43648240351708767])

## Example: modifying a matrix row in-place

In [11]:
function zero_row_v1(M::Matrix{N}, i) where {N}
    n = size(M, 2)
    M[i, :] = zeros(N, n)
    return M
end

function zero_row_v2(M::Matrix{N}, i) where {N}
    n = size(M, 2)
    for j in 1:n
        M[i, j] = zero(N)
    end
    return M
end

zero_row_v2 (generic function with 1 method)

In [14]:
M = rand(5, 5);

In [18]:
@btime zero_row_v1($M, 3)

  35.551 ns (1 allocation: 128 bytes)


5×5 Array{Float64,2}:
 0.775209  0.454217  0.231308  0.442315  0.051497
 0.78669   0.799652  0.301292  0.918232  0.40963
 0.0       0.0       0.0       0.0       0.0
 0.161415  0.577677  0.525344  0.493318  0.859636
 0.879471  0.514143  0.532165  0.169179  0.287497

In [19]:
@btime zero_row_v2($M, 3)

  4.132 ns (0 allocations: 0 bytes)


5×5 Array{Float64,2}:
 0.775209  0.454217  0.231308  0.442315  0.051497
 0.78669   0.799652  0.301292  0.918232  0.40963
 0.0       0.0       0.0       0.0       0.0
 0.161415  0.577677  0.525344  0.493318  0.859636
 0.879471  0.514143  0.532165  0.169179  0.287497

## Extending a matrix by columns

In [41]:
using ElasticArrays

In [54]:
A = ElasticArray(rand(3, 3))

3×3 ElasticArray{Float64,2,1,Array{Float64,1}}:
 0.79876   0.877491   0.0344291
 0.649965  0.0469924  0.585705
 0.188062  0.482117   0.734869

In [55]:
Ae = append!(A, rand(3))

3×4 ElasticArray{Float64,2,1,Array{Float64,1}}:
 0.79876   0.877491   0.0344291  0.259454
 0.649965  0.0469924  0.585705   0.282015
 0.188062  0.482117   0.734869   0.0461866

In [56]:
Ae[1, 1] = 100.0

100.0

In [57]:
A[1, 1]

100.0

---

In [47]:
A = rand(3, 3)

3×3 Array{Float64,2}:
 0.183724  0.903549  0.523527
 0.680694  0.986962  0.203901
 0.388022  0.158726  0.220619

In [49]:
Ae = hcat(A, rand(3))

3×4 Array{Float64,2}:
 0.183724  0.903549  0.523527  0.498343
 0.680694  0.986962  0.203901  0.217595
 0.388022  0.158726  0.220619  0.566324

In [50]:
Ae[1, 1] = 100.0

100.0

In [51]:
A[1, 1]

0.1837244232712345

## Implementation using Elastic Arrays

In [ ]:
function overapproximate(r::Rectification{N, ZT}, ::Type{<:Zonotope}) where {N, ZT<:AbstractZonotope{N}}
    Zout = r.X
    overapproximate!(r, Zout, Zonotope)
end

In [169]:
using ElasticArrays
using LazySets: _leq, isapproxzero

function _overapproximate(r::Rectification{N, <:Zonotope{N}}, ::Type{<:Zonotope}) where {N}
    Z = copy(r.X)
    c = center(Z)
    G = genmat(Z) |> ElasticArray
    n, m = size(G)
    H = overapproximate(Z, Hyperrectangle)

    @inbounds for i in 1:n
        lx, ux = H.center[i] - H.radius[i], H.center[i] + H.radius[i]

        if !_leq(lx, zero(N))
            nothing

        elseif _leq(ux, zero(N)) || isapproxzero(lx)
            c[i] = zero(N)
            for j in 1:m
                G[i, j] = zero(N)
            end

        else
            λ = ux / (ux - lx)
            μ = - λ * lx / 2
            c[i] = c[i] * λ + μ
            for j in 1:m
                G[i, j] = G[i, j] * λ
            end
            μvec = zeros(N, n)
            μvec[i] = μ
            append!(G, μvec)
        end
    end

    return Zonotope(c, G)
end

_overapproximate (generic function with 1 method)

In [81]:
Z = rand(Zonotope, dim=2)
r = Rectification(Z)
@btime _overapproximate($r, Zonotope);

  755.392 ns (16 allocations: 1.03 KiB)


In [82]:
Z = rand(Zonotope, dim=10)
r = Rectification(Z)
@btime _overapproximate($r, Zonotope);

  1.746 μs (24 allocations: 8.42 KiB)


In [83]:
Z = rand(Zonotope, dim=100)
r = Rectification(Z)
@btime _overapproximate($r, Zonotope);

  36.683 μs (116 allocations: 406.17 KiB)


In [85]:
Z = rand(Zonotope, dim=2)

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([0.9335253313176104, 1.1337002466448511], [-1.6906658703533577 1.105601863070022; 0.6885423150887638 1.2792172841083478])

In [86]:
r = Rectification(Z)
_overapproximate(r, Zonotope)

Zonotope{Float64,Array{Float64,1},ElasticArray{Float64,2,1,Array{Float64,1}}}([1.2437432349118058, 1.2220818512707083], [-1.1275447201756708 0.7373518121948244 0.6211532974586894 0.0; 0.5426187079158934 1.0081112150050882 0.0 0.3286480716502732])

In [87]:
Z

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([1.2437432349118058, 1.2220818512707083], [-1.6906658703533577 1.105601863070022; 0.6885423150887638 1.2792172841083478])

## Implementation using hcat

In [98]:
function _overapproximate_v2(r::Rectification{N, ZT}, ::Type{<:Zonotope}) where {N, ZT<:AbstractZonotope{N}}
    Z = copy(r.X)
    c = center(Z)
    G = genmat(Z)
    n, m = size(G)
    H = overapproximate(Z, Hyperrectangle)
    row_idx = Vector{Int}()
    μ_idx = Vector{N}()

    @inbounds for i in 1:n
        lx, ux = H.center[i] - H.radius[i], H.center[i] + H.radius[i]

        if !_leq(lx, zero(N))
            nothing

        elseif _leq(ux, zero(N)) || isapproxzero(lx)
            c[i] = zero(N)
            for j in 1:m
                G[i, j] = zero(N)
            end

        else
            λ = ux / (ux - lx)
            μ = - λ * lx / 2
            c[i] = c[i] * λ + μ
            for j in 1:m
                G[i, j] = G[i, j] * λ
            end
            push!(row_idx, i)
            push!(μ_idx, μ)
        end
    end

    q = length(row_idx)
    Gnew = zeros(N, n, q)
    j = 1
    for i in row_idx
        Gnew[i, j] = μ_idx[j]
        j += 1
    end
    
    return Zonotope(c, hcat(G, Gnew))
end

_overapproximate_v2 (generic function with 1 method)

In [170]:
Z = rand(Zonotope, dim=2)
r = Rectification(Z)

@btime _overapproximate_v2($r, Zonotope);
@btime _overapproximate($r, Zonotope);

  1.061 μs (24 allocations: 1.77 KiB)
  1.052 μs (22 allocations: 1.66 KiB)


In [171]:
Z = rand(Zonotope, dim=10)
r = Rectification(Z)

@btime _overapproximate_v2($r, Zonotope);
@btime _overapproximate($r, Zonotope);

  1.988 μs (28 allocations: 7.64 KiB)
  2.146 μs (30 allocations: 9.36 KiB)


In [172]:
Z = rand(Zonotope, dim=100)
r = Rectification(Z)

@btime _overapproximate_v2($r, Zonotope);
@btime _overapproximate($r, Zonotope);

  49.877 μs (38 allocations: 598.05 KiB)
  51.998 μs (123 allocations: 669.36 KiB)


In [173]:
Z = rand(Zonotope, dim=3)
r = Rectification(Z);

In [176]:
Z2 =  _overapproximate_v2(r, Zonotope);
Z1 =  _overapproximate(r, Zonotope);

In [177]:
Z1 == Z2

true

## In-place version

- We note that if the lazy rectification was built using an elastic array for the matrix of generators, then we can have an in-place version.

In [182]:
function _overapproximate_v2!(r::Rectification{N, ZT},
                               ::Type{<:Zonotope}
                             ) where {N, VT<:AbstractVector{N}, n, m, EA<:ElasticArray{N, n, m}, ZT<:Zonotope{N, VT, EA}}
    Z = r.X
    c = Z.center
    G = Z.generators
    H = overapproximate(Z, Hyperrectangle)

    @inbounds for i in 1:n
        lx, ux = H.center[i] - H.radius[i], H.center[i] + H.radius[i]

        if !_leq(lx, zero(N))
            nothing

        elseif _leq(ux, zero(N)) || isapproxzero(lx)
            c[i] = zero(N)
            for j in 1:m
                G[i, j] = zero(N)
            end

        else
            λ = ux / (ux - lx)
            μ = - λ * lx / 2
            c[i] = c[i] * λ + μ
            for j in 1:m
                G[i, j] = G[i, j] * λ
            end
            μvec = zeros(N, n)
            μvec[i] = μ
            append!(G, μvec)
        end
    end
    
    return Zonotope(c, G)
end

_overapproximate_v2! (generic function with 1 method)

In [191]:
Z = rand(Zonotope, dim=2)
Ze = Zonotope(Z.center, ElasticArray(Z.generators))
r = Rectification(Ze)

@btime _overapproximate_v2!($r, Zonotope);

  337.399 μs (21 allocations: 733.63 KiB)


In [190]:
Zout = _overapproximate_v2!(r, Zonotope)
Zout

Zonotope{Float64,Array{Float64,1},ElasticArray{Float64,2,1,Array{Float64,1}}}([1.3891136596350382, 0.25598291381429544], [-0.43972254574223 -0.3408112873996014 … 0.7323426964921975 0.0; 0.31287634927940217 0.9411955964352697 … 0.0 0.5211670880053866])